In [2]:
from vina import Vina

path = '../../vina'

v = Vina(sf_name='vina')

v.set_receptor(f'{path}/1iep_receptor.pdbqt')

v.set_ligand_from_file(f'{path}/1iep_ligand.pdbqt')
v.compute_vina_maps(center=[15.190, 53.903, 16.917], box_size=[20, 20, 20])

# Score the current pose
energy = v.score()
print('Score before minimization: %.3f (kcal/mol)' % energy[0])

# Minimized locally the current pose
energy_minimized = v.optimize()
print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
v.write_pose('1iep_ligand_minimized.pdbqt', overwrite=True)

# Dock the ligand
v.dock(exhaustiveness=32, n_poses=20)

Computing Vina grid ... Score before minimization: -12.513 (kcal/mol)
Score after minimization : -13.249 (kcal/mol)
done.
Performing local search ... done.
Performing docking (random seed: -1708539341) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [36]:
# %%time

# from vina import Vina

# path = '../../vina'

# v = Vina(sf_name='vina')

# v.set_receptor(f'{path}/1iep_receptor.pdbqt')

# v.set_ligand_from_file(f'{path}/1iep_ligand.pdbqt')
# v.compute_vina_maps(center=[15.190, 53.903, 16.917], box_size=[20, 20, 20])

# # Score the current pose
# energy = v.score()

In [55]:
from rdkit import Chem
from rdkit.Chem import AllChem
from openbabel import openbabel
import io

def smiles_to_pdbqt(smiles):
    
    # Convert SMILES to RDKit molecule
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        raise ValueError("Invalid SMILES string")
    
    # Generate 3D coordinates
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    AllChem.MMFFOptimizeMolecule(mol)
    
    # Compute Gasteiger charges
    AllChem.ComputeGasteigerCharges(mol)

    # Convert RDKit molecule to SDF format in memory
    sdf_stream = io.StringIO()
    writer = Chem.SDWriter(sdf_stream)
    writer.write(mol)
    writer.close()

    # Convert in-memory SDF to Open Babel molecule
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("sdf", "pdbqt")
    mol_ob = openbabel.OBMol()
    obConversion.ReadString(mol_ob, sdf_stream.getvalue())
    
    # Update Open Babel molecule with Gasteiger charges from RDKit molecule
    for atom, atom_ob in zip(mol.GetAtoms(), openbabel.OBMolAtomIter(mol_ob)):
        charge = atom.GetProp("_GasteigerCharge")
        atom_ob.SetPartialCharge(float(charge))

    pdbqt_data = obConversion.WriteString(mol_ob)
    
    return pdbqt_data

# Test
smiles_string = "CCO"  # Ethanol
pdbqt_data = smiles_to_pdbqt(smiles_string)
print(pdbqt_data)

REMARK  Name = 
REMARK  2 active torsions:
REMARK  status: ('A' for Active; 'I' for Inactive)
REMARK    1  A    between atoms: C_1  and  C_2
REMARK    2  A    between atoms: C_2  and  O_3
REMARK                            x       y       z     vdW  Elec       q    Type
REMARK                         _______ _______ _______ _____ _____    ______ ____
ROOT
ATOM      1  C   UNL     1      -0.888  -0.166   0.035  0.00  0.00    +0.034 C 
ATOM      2  C   UNL     1       0.488   0.436   0.226  0.00  0.00    +0.152 C 
ENDROOT
BRANCH   2   3
ATOM      3  O   UNL     1       1.486  -0.534  -0.057  0.00  0.00    -0.397 OA
ATOM      4  H   UNL     1       1.366  -0.817  -0.980  0.00  0.00    +0.210 HD
ENDBRANCH   2   3
TORSDOF 1



In [56]:
%load_ext autoreload
%autoreload 2

import rdkit
rdkit.Chem.Draw.IPythonConsole.ipython_maxProperties = -1

import dgym as dg

# load all data
print('load data')
path = '../../dgym-data'

deck = dg.MoleculeCollection.from_sdf(
    f'{path}/DSi-Poised_Library_annotated.sdf',
    reactant_names=['reagsmi1', 'reagsmi2', 'reagsmi3']
)

load data


In [60]:
deck[0].smiles

'c1ccc(SCCN2CCOCC2)cc1'

In [59]:
print(smiles_to_pdbqt(deck[0].smiles))

REMARK  Name = 
REMARK  4 active torsions:
REMARK  status: ('A' for Active; 'I' for Inactive)
REMARK    1  A    between atoms: C_4  and  S_5
REMARK    2  A    between atoms: S_5  and  C_6
REMARK    3  A    between atoms: C_6  and  C_7
REMARK    4  A    between atoms: C_7  and  N_8
REMARK                            x       y       z     vdW  Elec       q    Type
REMARK                         _______ _______ _______ _____ _____    ______ ____
ROOT
ATOM      1  C   UNL     1       0.165   0.270  -1.799  0.00  0.00    +0.090 C 
ENDROOT
BRANCH   1   2
ATOM      2  S   UNL     1       1.759   1.103  -2.152  0.00  0.00    -0.125 S 
BRANCH   2   6
ATOM      3  C   UNL     1       4.550  -0.894   0.889  0.00  0.00    +0.000 A 
ATOM      4  C   UNL     1       4.408   0.492   0.860  0.00  0.00    +0.001 A 
ATOM      5  C   UNL     1       3.555   1.088  -0.072  0.00  0.00    +0.015 A 
ATOM      6  C   UNL     1       2.832   0.300  -0.974  0.00  0.00    +0.007 A 
ATOM      7  C   UNL     1     